In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/LDS9_K273_BuiThiHuong_Cuoi_ky/'

/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/LDS9_K273_BuiThiHuong_Cuoi_ky


In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf

import time

from pyspark.sql.types import DoubleType
from pyspark.sql.functions import isnan, when, count, col, udf
from pyspark.sql.functions import *

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [5]:
# SparkContext.setSystemProperty('spark.executor.memory', '12g')
sc = SparkContext(master='local', appName='Recommendation_Amazon_Office_Products_fulldata')

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession(sc)

## Read data

In [8]:
start_time = time.time()

In [9]:
data = spark.read.csv('Data/ratings_Office_Products.csv')

In [ ]:
data.cache()

In [ ]:
data.show(5,truncate=True)

+--------------+----------+---+----------+
|           _c0|       _c1|_c2|       _c3|
+--------------+----------+---+----------+
|A2UESEUCI73CBO|0078800242|5.0|1374192000|
|A3BBNK2R5TUYGV|0113000316|5.0|1359417600|
| A5J78T14FJ5DU|0113000316|3.0|1318723200|
|A2P462UH5L6T57|043928631X|5.0|1356912000|
|A2E0X1MWNRTQF4|0439340039|1.0|1379721600|
+--------------+----------+---+----------+
only showing top 5 rows



In [ ]:
data.count()

1243186

In [10]:
data = data.withColumnRenamed('_c0','reviewerID')\
    .withColumnRenamed('_c1','asin')\
    .withColumnRenamed('_c2','overall')\
    .withColumnRenamed('_c3','unixReviewTime')

In [11]:
data_sub = data.select(['asin', 'overall', 'reviewerID'])

In [ ]:
data_sub.cache()

In [ ]:
data_sub.show(5)

+----------+-------+--------------+
|      asin|overall|    reviewerID|
+----------+-------+--------------+
|0078800242|    5.0|A2UESEUCI73CBO|
|0113000316|    5.0|A3BBNK2R5TUYGV|
|0113000316|    3.0| A5J78T14FJ5DU|
|043928631X|    5.0|A2P462UH5L6T57|
|0439340039|    1.0|A2E0X1MWNRTQF4|
+----------+-------+--------------+
only showing top 5 rows



In [ ]:
data_sub.count()

1243186

In [ ]:
data_sub.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: string (nullable = true)
 |-- reviewerID: string (nullable = true)



In [12]:
data_sub = data_sub.withColumn("overall", data_sub["overall"].cast(DoubleType()))

In [ ]:
data_sub.select([count(
                        when(col(c).isNull(), c))
                        .alias(c) 
                        for c in data_sub.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0


In [13]:
# Distinct users and movies
users = data_sub.select("reviewerID").distinct().count()
products = data_sub.select("asin").distinct().count()
numerator = data_sub.count()

In [ ]:
display(numerator, users, products)

1243186

909314

130006

In [ ]:
# Number of ratings matrix could contain if no empty cells
denominator = users * products
denominator

118216275884

In [ ]:
#Calculating sparsity
sparsity = 1 - (numerator*1.0 / denominator)
print ("Sparsity: "), sparsity

Sparsity: 


(None, 0.9999894838000039)

## Pipeline

In [ ]:
# Converting String to index
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [ ]:
# # Create an indexer
# indexer = StringIndexer(inputCol='asin', 
#                         outputCol='asin_idx')

# # Indexer identifies categories in the data
# indexer_model = indexer.fit(data_sub)

# # Indexer creates a new column with numeric index values
# data_indexed = indexer_model.transform(data_sub)

# # Repeat the process for the other categorical feature
# indexer1 = StringIndexer(inputCol='reviewerID', 
#                          outputCol='reviewerID_idx')
# indexer1_model = indexer1.fit(data_indexed)
# data_indexed = indexer1_model.transform(data_indexed)

In [ ]:
# data_indexed.show(5, truncate=True)

In [ ]:
# data_indexed.select([count(when(col(c).isNull(), c)).alias(c) for c in 
#            data_indexed.columns]).toPandas().T

In [ ]:
# # Smaller dataset so we will use 0.8 / 0.2
# (training, test) = data_indexed.randomSplit([0.8, 0.2])

In [18]:
indexer1 = StringIndexer(inputCol='asin',
                         outputCol='asin_idx', handleInvalid='skip')
indexer2 = StringIndexer(inputCol='reviewerID',
                         outputCol='reviewerID_idx', handleInvalid='skip')
als = ALS(maxIter=5,
          regParam=0.09,
          rank=25,
          userCol='reviewerID_idx',
          itemCol='asin_idx',
          ratingCol='overall',
          coldStartStrategy='drop',
          nonnegative=True)

In [19]:
pipeline_als = Pipeline(stages=[indexer1, indexer2, als])

## Train, test spliting

In [14]:
# smaller dataset so we will use 0.8/0.2
(training, test) = data_sub.randomSplit([0.8,0.2])

In [ ]:
training.count()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44777)
Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-55-83f5c40ad5a7>", line 1, in <module>
    training.count()
  File "/content/spark-2.4.0-bin-hadoop2.7/python/pyspark/sql/dataframe.py", line 522, in coun

TypeError: ignored

In [ ]:
test.count()

248124

In [ ]:
data.unpersist()

DataFrame[reviewerID: string, asin: string, overall: string, unixReviewTime: string]

In [ ]:
training.cache()

DataFrame[asin: string, overall: double, reviewerID: string]

In [ ]:
test.cache()

DataFrame[asin: string, overall: double, reviewerID: string]

## Build model

In [ ]:
# als = ALS(maxIter=5,
#           regParam=0.01,
#           userCol='userId',
#           itemCol='movieId',
#           ratingCol='rating',
#           coldStartStrategy='drop',
#           nonnegative=True)

In [ ]:
pipeline_als_model = pipeline_als.fit(training)

## Evaluation

In [ ]:
# evaluate the model by computing the RMSE on the test data
predictions = pipeline_als_model.transform(test)

In [ ]:
predictions.cache()

DataFrame[asin: string, overall: double, reviewerID: string, asin_idx: double, reviewerID_idx: double, prediction: float]

In [ ]:
predictions.show(5)

+----------+-------+--------------+--------+--------------+----------+
|      asin|overall|    reviewerID|asin_idx|reviewerID_idx|prediction|
+----------+-------+--------------+--------+--------------+----------+
|B0036TGGVO|    5.0|A180V4L3DDW9W2|   148.0|      258696.0|0.60845494|
|B0036TGGVO|    5.0|A134OXKUUA2N57|   148.0|       25324.0| 1.6193192|
|B0036TGGVO|    5.0|A3GPQYXA5KNPBI|   148.0|      359168.0|0.95481056|
|B0036TGGVO|    5.0|A2AZ4VVLC8FMCO|   148.0|      563855.0| 1.6018285|
|B0036TGGVO|    5.0|A13BX9O5UDBILC|   148.0|         604.0| 2.5201797|
+----------+-------+--------------+--------+--------------+----------+
only showing top 5 rows



In [ ]:
evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol='overall',
                                predictionCol='prediction')

In [ ]:
rmse = evaluator.evaluate(predictions)

In [ ]:
rmse

2.2480579877951743

In [ ]:
data.describe().show()

+-------+--------------------+--------------------+------------------+--------------------+
|summary|          reviewerID|                asin|           overall|      unixReviewTime|
+-------+--------------------+--------------------+------------------+--------------------+
|  count|             1243186|             1243186|           1243186|             1243186|
|   mean|                null|2.0396489659421234E9|3.9790208383942547|1.3436354877020817E9|
| stddev|                null| 2.374136541035274E9|1.4179903355832062| 6.759409513579747E7|
|    min|A00001483M88NBD66...|          0078800242|               1.0|          1000080000|
|    max|       AZZZZXVAOWWME|          BT008G9O9A|               5.0|           999993600|
+-------+--------------------+--------------------+------------------+--------------------+



* RMSE = 1.064 > stddev = 0.93 => This model is not goood.

In [ ]:
data.agg({'overall': 'mean'}).show()

+------------------+
|      avg(overall)|
+------------------+
|3.9790208383942547|
+------------------+



In [ ]:
data.agg({'overall': 'std'}).show()

+------------------+
|   stddev(overall)|
+------------------+
|1.4179903355832062|
+------------------+



In [ ]:
predictions.unpersist()

DataFrame[asin: string, overall: double, reviewerID: string, asin_idx: double, reviewerID_idx: double, prediction: float]

## Turning parameters

In [20]:
als_t = ALS(maxIter=20,
          regParam=0.1,
          userCol='reviewerID_idx',
          itemCol='asin_idx',
          ratingCol='overall',
          coldStartStrategy='drop',
          nonnegative=True)
# if the capacity of computer is power enough => User RandomSearchCV or GridSearchCV
# if not, do manually one by one

In [21]:
pipeline_als_t = Pipeline(stages=[indexer1, indexer2, als_t])

In [22]:
pipeline_als_model_t = pipeline_als_t.fit(training)

In [23]:
# evaluate the model by computing the RMSE on the test data
predictions_t = pipeline_als_model_t.transform(test)

In [ ]:
predictions_t.cache()

DataFrame[asin: string, overall: double, reviewerID: string, asin_idx: double, reviewerID_idx: double, prediction: float]

In [ ]:
predictions_t.show(5)

+----------+-------+--------------+--------+--------------+----------+
|      asin|overall|    reviewerID|asin_idx|reviewerID_idx|prediction|
+----------+-------+--------------+--------+--------------+----------+
|B00AVWKUJI|    3.0|A2P6FU1HQ1OFAP|   148.0|        8145.0| 3.7511604|
|B00AVWKUJI|    5.0|A14EF1PPKMSEPU|   148.0|        4992.0| 2.0782158|
|B00AVWKUJI|    5.0|A2MEXWR4GQIHQ9|   148.0|       71029.0|  3.989509|
|B00AVWKUJI|    5.0|A2RLL7BESPS3RR|   148.0|       45376.0| 2.9472685|
|B00AVWKUJI|    5.0|A1I72CX1S7ZA1L|   148.0|      124906.0| 3.8806462|
+----------+-------+--------------+--------+--------------+----------+
only showing top 5 rows



In [ ]:
rmse_t = evaluator.evaluate(predictions_t)

In [ ]:
rmse_t

1.7793128874143782

In [ ]:
data.describe().show()

+-------+--------------------+--------------------+------------------+--------------------+
|summary|          reviewerID|                asin|           overall|      unixReviewTime|
+-------+--------------------+--------------------+------------------+--------------------+
|  count|             1243186|             1243186|           1243186|             1243186|
|   mean|                null|2.0396489659421234E9|3.9790208383942547|1.3436354877020817E9|
| stddev|                null| 2.374136541035274E9|1.4179903355832062| 6.759409513579747E7|
|    min|A00001483M88NBD66...|          0078800242|               1.0|          1000080000|
|    max|       AZZZZXVAOWWME|          BT008G9O9A|               5.0|           999993600|
+-------+--------------------+--------------------+------------------+--------------------+



* RMSE = 0.998 while stddev = 0.93 => This model is better.

* pipeline_als_model_t is better than pipeline_als because rmse_t is lower

In [ ]:
predictions_t.unpersist()

DataFrame[asin: string, overall: double, reviewerID: string, asin_idx: double, reviewerID_idx: double, prediction: float]

## Save the results

In [ ]:
training.unpersist()

DataFrame[asin: string, overall: double, reviewerID: string]

In [ ]:
test.unpersist()

DataFrame[asin: string, overall: double, reviewerID: string]

### create user_recs

In [24]:
model = pipeline_als_model_t.stages[-1]

In [25]:
# get 10 recommendations which have highest rating
user_recs = model.recommendForAllUsers(users)

In [26]:
user_recs.cache()

DataFrame[reviewerID_idx: int, recommendations: array<struct<asin_idx:int,rating:float>>]

In [ ]:
user_recs.printSchema()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: ignored

In [ ]:
# for user in user_recs.head(3):
#   print(user)

In [ ]:
# user_recs.show(3, truncate=False)

In [ ]:
# user_recs.count()

### create asin_data

In [ ]:
pipeline_als_model_t.stages[0:3]

[StringIndexer_3113d61f0509, StringIndexer_e49ccd700e9c, ALS_441a3374cab8]

In [ ]:
# check to ensure the asin_data contains all reviewerIDs
# transform "training" data by fitted indexer model
asin_data_train = pipeline_als_model_t.stages[0].transform(training).select('asin','asin_idx').distinct()
asin_data_train = asin_data_train.withColumnRenamed('asin_idx', 'asin_idx_train')
asin_data_train.cache()
asin_data_train.count()
# transform "data" by fitted indexer model
asin_data_full = pipeline_als_model_t.stages[0].transform(data).select('asin','asin_idx').distinct()
asin_data_full = asin_data_full.withColumnRenamed('asin_idx', 'asin_idx_full')
asin_data_full.cache()
asin_data_full.count()

combined_asin_data = asin_data_full.join(asin_data_train, on='asin')
combined_asin_data.show(5)
# coompare
combined_asin_data = combined_asin_data.withColumn('check', col('asin_idx_full')==col('asin_idx_train'))
combined_asin_data.filter(col('check') != 'true').show()

asin_data_train.unpersist()
asin_data_full.unpersist()

+----------+-------------+--------------+
|      asin|asin_idx_full|asin_idx_train|
+----------+-------------+--------------+
|143972363X|     103016.0|      103016.0|
|1564516458|       5938.0|        5938.0|
|1574896741|     116098.0|      116098.0|
|1619921685|      47349.0|       47349.0|
|1622261038|      73635.0|       73635.0|
+----------+-------------+--------------+
only showing top 5 rows

+----+-------------+--------------+-----+
|asin|asin_idx_full|asin_idx_train|check|
+----+-------------+--------------+-----+
+----+-------------+--------------+-----+



DataFrame[asin: string, asin_idx_full: double]

In [ ]:
# final asin_data
asin_data = pipeline_als_model_t.stages[0].transform(data).select('asin','asin_idx').distinct()
asin_data.cache()

DataFrame[asin: string, asin_idx: double]

In [ ]:
asin_data.show(5)

+----------+--------+
|      asin|asin_idx|
+----------+--------+
|143972363X|103016.0|
|1564516458|  5938.0|
|1574896741|116098.0|
|1619921685| 47349.0|
|1622261038| 73635.0|
+----------+--------+
only showing top 5 rows



### create reviewer_data

In [ ]:
pipeline_als_model_t.stages[0:3]

[StringIndexer_3113d61f0509, StringIndexer_e49ccd700e9c, ALS_441a3374cab8]

In [ ]:
# check to ensure the reviewerID_data contains all reviewerIDs
# transform "training" data by fitted indexer model
reviewerID_data_train = pipeline_als_model_t.stages[1].transform(training).select('reviewerID','reviewerID_idx').distinct()
reviewerID_data_train = reviewerID_data_train.withColumnRenamed('reviewerID_idx', 'reviewerID_idx_train')
reviewerID_data_train.cache()
reviewerID_data_train.count()
# transform "data" by fitted indexer model
reviewerID_data_full = pipeline_als_model_t.stages[1].transform(data).select('reviewerID','reviewerID_idx').distinct()
reviewerID_data_full = reviewerID_data_full.withColumnRenamed('reviewerID_idx', 'reviewerID_idx_full')
reviewerID_data_full.cache()
reviewerID_data_full.count()

combined_reviewerID_data = reviewerID_data_full.join(reviewerID_data_train, on='reviewerID')
combined_reviewerID_data.show(5)
# coompare
combined_reviewerID_data = combined_reviewerID_data.withColumn('check', col('reviewerID_idx_full')==col('reviewerID_idx_train'))
combined_reviewerID_data.filter(col('check') != 'true').show()

reviewerID_data_train.unpersist()
reviewerID_data_full.unpersist()

+--------------------+-------------------+--------------------+
|          reviewerID|reviewerID_idx_full|reviewerID_idx_train|
+--------------------+-------------------+--------------------+
|A06616661UYTZF0LZ...|           536991.0|            536991.0|
|A06661002R54BLOVW...|           449627.0|            449627.0|
|A09775342U0HABE80...|           664102.0|            664102.0|
|      A1016H15N3HBYY|           631082.0|            631082.0|
|A1020707QSDKPEC1Z8IH|           532187.0|            532187.0|
+--------------------+-------------------+--------------------+
only showing top 5 rows

+----------+-------------------+--------------------+-----+
|reviewerID|reviewerID_idx_full|reviewerID_idx_train|check|
+----------+-------------------+--------------------+-----+
+----------+-------------------+--------------------+-----+



DataFrame[reviewerID: string, reviewerID_idx_full: double]

In [ ]:
# final reviewerID_data
reviewerID_data = pipeline_als_model_t.stages[1].transform(data).select('reviewerID','reviewerID_idx').distinct()
reviewerID_data.cache()

DataFrame[reviewerID: string, reviewerID_idx: double]

In [ ]:
reviewerID_data.show(5)

+--------------+--------------+
|    reviewerID|reviewerID_idx|
+--------------+--------------+
|A3LLEX1R9IXQSO|       95329.0|
| A58MHEM08DXVC|      375743.0|
| AUO5VAEMF7VWQ|      117164.0|
|A1VDGF6ZMK15L2|       51991.0|
|A2TGN187KKRIIB|      110679.0|
+--------------+--------------+
only showing top 5 rows



In [27]:
# save parquets
user_recs.write.parquet('C5_Recommendation_Amazon_Office_Products_user_recs_fulldata.parquet', mode='overwrite')

Py4JJavaError: ignored

In [ ]:
reviewerID_data.write.parquet('C5_Recommendation_Amazon_Office_Products_reviewerID_fulldata.parquet', mode='overwrite')

In [ ]:
asin_data.write.parquet('C5_Recommendation_Amazon_Office_Products_asin_fulldata.parquet', mode='overwrite')

## Recomendation for specific users

In [ ]:
# test recommendation by a given user
reviewerId = 'A3CJ7MHAS9IMAM'

In [ ]:
reviewerId_idx = reviewerID_data.filter(col('reviewerID')==reviewerId).select('reviewerID_idx').collect()[0][0]
reviewerId_idx

23.0

In [ ]:
result = user_recs.filter(user_recs['reviewerID_idx']==reviewerId_idx)
result.show(truncate=False)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45431)
Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

In [ ]:
result = result.select(result.reviewerID_idx, explode(result.recommendations))
result.show(5)

In [ ]:
# recommendation result
result = result.withColumn('asin_idx', result.col.getField('asin_idx'))\
                           .withColumn('rating', result.col.getField('rating'))
result = result.join(reviewerID_data, on = 'reviewerID_idx').join(asin_data, on='asin_idx')
result.show()

In [ ]:
# create a function to do recommendation
def recommendation_specific_user(userId):
  reviewerId = userId
  reviewerId_idx = reviewerID_data.filter(col('reviewerID')==reviewerId).select('reviewerID_idx').collect()[0][0]
  result = user_recs.filter(user_recs['reviewerID_idx']==reviewerId_idx)
  result = result.select(result.reviewerID_idx, explode(result.recommendations))
  result = result.withColumn('asin_idx', result.col.getField('asin_idx'))\
                           .withColumn('rating', result.col.getField('rating'))
  result = result.join(reviewerID_data, on = 'reviewerID_idx').join(asin_data, on='asin_idx')
  return result

In [ ]:
# make recommendations to some users: A3CJ7MHAS9IMAM, A3LGT6UZL99IW1, A21KNRUAA5RK5E
lst_user = ['A3CJ7MHAS9IMAM','A3LGT6UZL99IW1', 'A21KNRUAA5RK5E']
for i in lst_user:
  print('Recommendations for user', i, ':')
  recommendation_specific_user(i).show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45431)
Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Recommendations for user A3CJ7MHAS9IMAM :


Py4JNetworkError: ignored

## Recomendation by threshold

In [ ]:
# # filter all movieIds having rationg >= 3.0
# result.filter(result.rating >= 3.0).show()

## Save results

In [ ]:
# save parquet
user_recs.write.parquet('C5_Recommendation_Amazon_Office_Products_user_recs_fulldata.parquet', mode='overwrite')
reviewerID_data.write.parquet('C5_Recommendation_Amazon_Office_Products_reviewerID_fulldata.parquet', mode='overwrite')
asin_data.write.parquet('C5_Recommendation_Amazon_Office_Products_asin_fulldata.parquet', mode='overwrite')

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45431)
Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

In [ ]:
user_recs.unpersist()
reviewerID_data.unpersist()
asin_data.unpersist()

In [ ]:
# read parquet
user_recs = spark.read.parquet('C5_Recommendation_Amazon_Office_Products_user_recs_fulldata.parquet')
reviewerID_data = spark.read.parquet('C5_Recommendation_Amazon_Office_Products_reviewerID_fulldata.parquet')
asin_data = spark.read.parquet('C5_Recommendation_Amazon_Office_Products_asin_fulldata.parquet')

In [ ]:
user_recs.cache()
reviewerID_data.cache()
asin_data.cache()

In [ ]:
lst_user = ['AZZ56WF4X19G2', 'A335QXPTV1RIV1', 'ATIMW8SYGAASW']
for i in lst_user:
  print('Recommendations for user', i, ':')
  recommendation_specific_user(i).show()

In [ ]:
user_recs.cache()
reviewerID_data.cache()
asin_data.cache()

In [ ]:
end_time = time.time()

In [ ]:
duration = (end_time - start_time)/60
duration